In [ ]:
import sys
import os
import pandas as pd
from sqlalchemy import create_engine, text
sys.path.append(os.path.abspath(".."))
from db.db_config import ORACLE_CONFIG

from sqlalchemy import create_engine, text
from db.db_config import ORACLE_CONFIG
# Create the engine using correct PDB DSN
engine = create_engine(
    f"oracle+oracledb://{ORACLE_CONFIG['user']}:{ORACLE_CONFIG['password']}@{ORACLE_CONFIG['dsn']}"
)

with engine.connect() as conn:
    print(" Connected as:", conn.execute(text("SELECT user FROM dual")).scalar())
    print(" PDB Name:", conn.execute(text("SELECT sys_context('userenv', 'con_name') FROM dual")).scalar())

    print("\n Checking if BANK_REVIEWS table exists in MY schema...\n")

    result = conn.execute(text("""
        SELECT object_name, object_type, status 
        FROM user_objects 
        WHERE object_name = 'BANK_REVIEWS'
    """)).fetchall()

    if result:
        for row in result:
            print(f" Found: {row.object_name}, Type: {row.object_type}, Status: {row.status}")
    else:
        print(" No table named BANK_REVIEWS found in current schema.")

    print("\n Trying SELECT COUNT(*) FROM BANK_REVIEWS directly...\n")

    try:
        count = conn.execute(text("SELECT COUNT(*) FROM BANK_REVIEWS")).scalar()
        print(f" Success! Row count: {count}")
    except Exception as e:
        print(f" Failed to access table directly: {e}")



 Connected as: SYSTEM
 PDB Name: XEPDB1

 Checking if BANK_REVIEWS table exists in MY schema...

 Found: BANK_REVIEWS, Type: TABLE, Status: VALID

 Trying SELECT COUNT(*) FROM BANK_REVIEWS directly...

 Success! Row count: 849


In [3]:
df = pd.read_sql("SELECT * FROM system.bank_reviews", con=engine)
for col in df.columns:
    if df[col].apply(lambda x: hasattr(x, 'read')).any():
        df[col] = df[col].apply(lambda x: x.read() if hasattr(x, 'read') else x)

df.head()

,review,rating,review_date,bank,source,sentiment_label,sentiment_score
0,A great app. It's like carrying a bank in your...,4,2025-06-07,Commercial Bank of Ethiopia,Google Play,POSITIVE,0.999293
1,More than garrantty bank EBC.,4,2025-06-07,Commercial Bank of Ethiopia,Google Play,POSITIVE,0.997196
2,really am happy to this app it is Siple to use...,5,2025-06-07,Commercial Bank of Ethiopia,Google Play,POSITIVE,0.998870
3,I liked this app. But the User interface is ve...,2,2025-06-07,Commercial Bank of Ethiopia,Google Play,NEGATIVE,0.999684
4,"""Why don’t your ATMs support account-to-accoun...",4,2025-06-06,Commercial Bank of Ethiopia,Google Play,NEGATIVE,0.996465
